In [5]:
from __future__ import print_function
import sys

# This is not required if you've installed pycparser into
# your site-packages/ with setup.py
#
sys.path.extend(['.', '..'])

from pycparser import c_parser, c_ast
from pycparser import parse_file

In [49]:
ast = parse_file('./pycparser-master/examples/c_files/hash.c', use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include']
        )

'''
ast = parse_file('./pycparser-master/examples/c_files/year.c', use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include']
        )'''

"\nast = parse_file('./pycparser-master/examples/c_files/year.c', use_cpp=True,\n        cpp_path='gcc',\n        cpp_args=['-E', r'-Iutils/fake_libc_include']\n        )"

In [50]:
ast.show(attrnames=True, nodenames=True)

FileAST: 
  Typedef <ext[0]>: name=ReturnCode, quals=[], storage=['typedef']
    TypeDecl <type>: declname=ReturnCode, quals=[]
      Enum <type>: name=tagReturnCode
        EnumeratorList <values>: 
          Enumerator <enumerators[0]>: name=SUCCESS
          Enumerator <enumerators[1]>: name=FAIL
  Typedef <ext[1]>: name=Entry, quals=[], storage=['typedef']
    TypeDecl <type>: declname=Entry, quals=[]
      Struct <type>: name=tagEntry
        Decl <decls[0]>: name=key, quals=[], storage=[], funcspec=[]
          PtrDecl <type>: quals=[]
            TypeDecl <type>: declname=key, quals=[]
              IdentifierType <type>: names=['char']
        Decl <decls[1]>: name=value, quals=[], storage=[], funcspec=[]
          PtrDecl <type>: quals=[]
            TypeDecl <type>: declname=value, quals=[]
              IdentifierType <type>: names=['char']
  Typedef <ext[2]>: name=Node, quals=[], storage=['typedef']
    TypeDecl <type>: declname=Node, quals=[]
      Struct <type>: name=tagN

In [56]:
operators = {}
operands = {}
# broj pokazivaca za pointerdecl. anulira se u identifiertype
num_pointers = 0
# ne zanima me ID ako je deklaracija, inace me zanima
is_declaration = 0
# pozivi funkcija
func_call = 0

In [57]:
def traverse(node):
    global num_pointers
    global is_declaration
    global func_call
    #print(node)

    
    '''
        pycparser prepoznaje 46 tipova cvorova stabla AST
    
        Potrebno je obici cvorove stabla i obraditi ih na odgovarajuci nacin
    '''
    # child od FileAST moze da bude Decl|FuncDef|Typedef
    
    if isinstance (node, c_ast.ArrayDecl):
        print("ArrayDecl")
        # ArrayDecl[type*, dim*, dim_quals*]
        
        if node.type:
            traverse(node.type)
        if node.dim:
            traverse(node.dim)
        if node.dim_quals:
            traverse(node.dim_quals)
        
    elif isinstance (node, c_ast.ArrayRef):
        print("ArrayRef")
        # ArrayRef[name*, subscript*]
        
        if node.name:
            traverse(node.name)
        if node.subscript:
            traverse(node.subscript)
        
    elif isinstance (node, c_ast.Assignment):
        print("Assignment")
        # Assignment[op, lvalue*, rvalue*]
        
        if node.op in operators:
            operators[node.op] += 1
        else:
            operators[node.op] = 1
        
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1
                        
        if node.lvalue:
            traverse(node.lvalue)
        if node.rvalue:
            traverse(node.rvalue)
                
        
    elif isinstance (node, c_ast.BinaryOp):
        print("Binary op")
        # BinaryOp[op, left*, right*]
        # a = 5
        # a = b = i + 5 - l/3
        
        if node.op in operators:
            operators[node.op] += 1
        else: 
            operators[node.op] = 1
            
        # moraju da postoje left i right, provera je suvisna
        if node.left:
            traverse(node.left)
        if node.right:
            traverse(node.right)
        
        
    elif isinstance (node, c_ast.Break):
        print("Break")
        # Break[]
        
        if "break" in operators:
            operators["break"] += 1
        else:
            operators["break"] = 1
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1            
        
    elif isinstance (node, c_ast.Case):
        print("Case")
        # Case[expr*, stmts**]
        
        # da li se dodaje ? ['case' i ':']
        
        if node.expr:
            traverse(node.expr)
        if node.stmts:
            for stmt in node.stmts:
                traverse(stmt)
        
        
    elif isinstance (node, c_ast.Cast):
        print("Cast")
        # Cast[to_type*, expr*] -> (int) l
        if '(' in operators:
            operators['('] += 1
        else:
            operators['('] = 1
        if ')' in operators:
            operators[')'] += 1
        else:
            operators[')'] = 1
        
        if node.to_type:
            traverse(node.to_type)
        if node.expr:
            traverse(node.expr)
        
    elif isinstance (node, c_ast.Compound):
        print("Compound")
        # Compound[block_items**]
        # ->  {
        #       ..., ..., 
        #       ...
        #      }
        print('{')
        
        if '{' in operators:
            operators['{'] += 1
        else:
            operators['{'] = 1
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1
        
        if node.block_items:
            for item in node.block_items:
                traverse(item)
        
        print('}')
        
    elif isinstance (node, c_ast.CompoundLiteral):
        print("Compound literal")
        # CompoundLiteral[type*, init*]
        
        if node.type:
            traverse(node.type)
        if node.init:
            traverse(node.init)
        
    elif isinstance (node, c_ast.Constant):
        print("Constant")
        # Const[type, value]
        # U ovom slucaju ne treba nista da se radi jer je konstanta karakter 'a' ili broj 5, ...
        return
    elif isinstance (node, c_ast.Continue):
        print("Continue")
        #Continue[]
        
        if "continue" in operators:
            operators["continue"] += 1
        else:
            operators["continue"] = 1
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1  
        
    elif isinstance (node, c_ast.Decl):
        print("Decl")
        # Decl[name, quals, storage, funspace, type*, init*, bitsize*]
        
        # name parametar nije znacajan zato sto imena funkcija i promenljivih prilikom deklaracija
        # ne spadaju u operatore tj. operande
        
        is_declaration += 1
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                #print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print(quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                
        # extern, register
        storage = node.storage        
        if storage: # if true
            if (len(storage) == 2):
                #print(storage[0]+"   "+storage[1])
                if (storage[0] in operators):
                    operators[storage[0]] += 1
                else:
                    operators[storage[0]] = 1
                if (storage[1] in operators):
                    operators[storage[1]] += 1
                else:
                    operators[storage[1]] = 1
            else: #(len(storage) == 1):
                #print(storage[0])
                if (storage[0] in operators):
                    operators[storage[0]] += 1
                else:
                    operators[storage[0]] = 1   
        
        # funspec
        funcspec = node.funcspec
        if funcspec:
            print(funcspec[0])
            if (funcspec[0] in operators):
                operators[funcspec[0]] += 1
            else:
                operators[funcspec[0]] = 1  
        
        if node.type:
            traverse(node.type)
        if node.init:
            if '=' in operators:
                operators['='] += 1
            else:
                operators['='] = 1
            traverse(node.init)
        if node.bitsize:
            traverse(node.bitsize)
            
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1
        
        is_declaration -= 1
                    
    elif isinstance (node, c_ast.DeclList):
        print("decl list")
        # DeclList[decls**]
        
        if node.decls:
            for decl in node.decls:
                traverse(decl)
        
    elif isinstance (node, c_ast.Default):
        print("default")
        # Default[stmts**]
        
        if node.stmts:
            for stmt in node.stmts:
                traverse(stmt)
        
    elif isinstance (node, c_ast.DoWhile):
        print("do while")
        # DoWhile[cond*, stmt*]
        '''
        do {
           statement(s);
        } while( condition );
        '''
        
        if "do while" in operators:
            operators["do while"] += 1
        else:
            operators["do while"] = 1
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1 
        if '{' in operators:
            operators['{'] += 1
        else:
            operators['{'] = 1
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1  
        
        if node.cond:
            traverse(node.cond)
        if node.stmt:
            traverse(node.stmt)        
        
    elif isinstance (node, c_ast.EllipsisParam):
        print("ellipsis param")
        # EllipsisParam[]
        # ( ... ) deo u deklaraciji funkcije
        # treba samo zabeleziti zagrade ili ne raditi nista ako se zagrade broje naknadno
        
    elif isinstance (node, c_ast.EmptyStatement):
        print("empty statement")
        # EmptyStatement[]
        # samo ;
        
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1 
        
    elif isinstance (node, c_ast.Enum):
        print("enum")
        # Enum[name, values*]
        # enum week{Mon, Tue, Wed, Thur, Fri, Sat, Sun};
        
        # Enumeration type specifier
        # name: an optional ID
        # values: an EnumeratorList
        
        if "enum" in operators:
            operators["enum"] += 1
        else:
            operators["enum"] = 1
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1 
        if '{' in operators:
            operators['{'] += 1
        else:
            operators['{'] = 1
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1  
        
        # posto je u pitanju deklaracija ime i vrednosti nas ne zanimaju
        
        if node.values:
            traverse(node.values)        
        
    elif isinstance (node, c_ast.Enumerator):
        print("Enumerator: DODAJ AKCIJE")
        # Enumerator[name, values*]
        # A name/value pair for enumeration values
        
        
        
        
    elif isinstance (node, c_ast.EnumeratorList):
        print("Enumerator list")
        # EnumeratorList[enumerators**]
        # A list of enumerators
        
        if node.enumerators:
            for enumerator in node.enumerators:
                traverse(enumerator)
        
    elif isinstance (node, c_ast.ExprList):
        print("Expr list")
        # ExprList[exprs**]
        # izrazi razdvojeni zarezom
        
        if node.exprs:
            l = len(node.exprs)
            for expr in node.exprs:
                traverse(expr)
        
        for i in range(1,l):
            if ',' in operators:
                operators[','] += 1
            else:
                operators[','] = 1 
        
    elif isinstance (node, c_ast.For):
        print("For")
        # for (init; cond; next) stmt
        # For: [init*, cond*, next*, stmt*]
        # () se ne racuna
        
        if "for" in operators:
            operators["for"] += 1
        else:
            operators["for"] = 1  
        if '{' in operators:
            operators['{'] += 1
        else:
            operators['{'] = 1  
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1  
        
        if node.init:
            traverse(node.init)
        if node.cond:
            traverse(node.cond)
        if node.next:
            traverse(node.next)
        if node.stmt:
            traverse(node.stmt)
        
    elif isinstance (node, c_ast.FuncCall):
        print("Func call")
        # FuncCall[name*, args*]
        # name: Id
        # args: ExprList
        # postoji nacin da se prebroje svi pozivi funkcija pa mozda ne mora ovde
        
        func_call = 1
        
        if node.name:
            traverse(node.name)
        if node.args:
            traverse(node.args)
        
        # ne treba zaboraviti na zagrade ( i )
        
    elif isinstance (node, c_ast.FuncDecl):
        print("Func decl") 
        # FuncDecl[args*, type*]
        if node.args:
            traverse(node.args)
        if node.type:
            traverse(node.type)
        
    elif isinstance (node, c_ast.FuncDef):
        print("FuncDef")
        # FuncDef [decl*, param_decls**, body*]
        node.decl.show(attrnames=True, nodenames=True)
        
        # oduzimamo ; jer ne postoji, a dodaje se u Decl koji se nalazi u okviru FuncDef
        if ';' in operators:
            operators[';'] -= 1
        else:
            operators[';'] = -1
        
        if node.decl:
            traverse(node.decl)
        if node.param_decls:
            for pd in node.param_decls:
                if pd:
                    traverse(pd)
        if node.body:
            traverse(node.body)
        
        # node.show(attrnames=True, nodenames=True)
        
        print('-----------------')
    elif isinstance (node, c_ast.Goto):
        print("goto")
        # Goto[name]
        # goto label;
        
        if "goto" in operators:
            operators["goto"] -= 1
        else:
            operators["goto"] = -1
        
        if node.name:
            if node.name in operators:
                operators[node.name] -= 1
            else:
                operators[node.name] = -1
        
        if ';' in operators:
            operators[';'] -= 1
        else:
            operators[';'] = -1
        
    elif isinstance (node, c_ast.ID):
        print("ID")
        # ID[name]
        
        if not (is_declaration) or func_call:
            if node.name in operands:
                operands[node.name] += 1
            else:
                operands[node.name] = 1
            func_call = 0
        
        return
        
    elif isinstance (node, c_ast.IdentifierType):
        print("Identifier Type")
        # IdentifierType[names] [long long int]
        
        id_type = ""
        if node.names:
            for name in node.names:
                id_type += name
                id_type += ' '
                
        # dodajem * koliko ima num_pointers
        for i in range(0,num_pointers):
            id_type += '*'
                
        if id_type in operands:
            operands[id_type] += 1
        else: 
            operands[id_type] = 1
            
        num_pointers = 0
        
    elif isinstance (node, c_ast.If):
        print("if")
        # If: [cond*, iftrue*, iffalse*]
        # if (cond*)
        # {
        #   iftrue*
        # }
        # iffalse* -> else {}
        
        if "if" in operators:
            operators["if"] += 1
        else:
            operators["if"] = 1  
        if '{' in operators:
            operators['{'] += 1
        else:
            operators['{'] = 1  
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1
        
        if node.cond:
            traverse(node.cond)
        if node.iftrue:
            traverse(node.iftrue)
        
        if node.iffalse:
            if "else" in operators:
                operators["else"] += 1
            else:
                operators["else"] = 1  
            if '{' in operators:
                operators['{'] += 1
            else:
                operators['{'] = 1  
            if '}' in operators:
                operators['}'] += 1
            else:
                operators['}'] = 1  
            traverse(node.iffalse)
        
    elif isinstance (node, c_ast.InitList):
        print("init list")
        # InitList: [exprs**]
        if node.exprs:
            for expr in node.exprs:
                traverse(expr)
        
        
    elif isinstance (node, c_ast.Label):
        print("label")
        # Label[name, stmt*]
        
        if ':' in operators:
                operators[':'] += 1
        else:
            operators[':'] = 1  
        
        if node.stmt:
            traverse(node.stmt)
        
    elif isinstance (node, c_ast.NamedInitializer):
        print("Named Initializer ?")
        # NamedInitializer[name**, expr*]
        
        if node.name:
            for n in node.name:
                traverse(n)
        if node.expr:
            traverse(node.expr)        
        
    elif isinstance (node, c_ast.ParamList):
        print("Param list")
        # ParamList[params**]
        if node.params:
            for param in node.params:
                print(param)
                if param:
                    traverse(param)
        '''
        ParamList <args>: 
          Typename <params[0]>: name=None, quals=[]
            TypeDecl <type>: declname=None, quals=[]
              IdentifierType <type>: names=['void']
        '''
    elif isinstance (node, c_ast.Pragma):
        print("Pragma")
        # Pragma[string]
        
        if "pragma" in operators:
            operators["pragma"] += 1
        else:
            operators["pragma"] = 1
        
    elif isinstance (node, c_ast.PtrDecl):
        print("PtrDecl")
        # PtrDecl[quals, type*]
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                #print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print(quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                    
        num_pointers += 1
        
        if node.type:
            traverse(node.type)
        
    elif isinstance (node, c_ast.Return):
        print("Return")
        # Return[expr*]
        if "return" in operators:
            operators["return"] += 1
        else:
            operators["return"] = 1
            
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1
        
        if node.expr:
            traverse(node.expr)
        
    elif isinstance (node, c_ast.Struct):
        print("struct")                 
        # Struct[name, decls**]
        '''
            struct Books {
               char  title[50];
               char  author[50];
               char  subject[100];
               int   book_id;
            } book; 
        '''
        if "struct" in operators:
            operators["struct"] += 1
        else:
            operators["struct"] = 1
        # sa struct definisemo tip. ime tipa je operand
        # mozda ne treba da bude operand posto je deklaracija
        '''
        if node.name:
            if node.name in operands:
                operands[node.name] += 1
            else:
                operands[node.name] = 1
        '''        
        if '{' in operators:
                operators['{'] += 1
        else:
            operators['{'] = 1  
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1 
        
        if node.decls:
            for decl in node.decls:
                traverse(decl)
        
    elif isinstance (node, c_ast.StructRef):
        print("Struct ref")
        # type: . or ->
        # name.field or name->field
        # StructRef: [name*, type, field*]
        
        if node.type:
            if node.type in operators:
                operators[node.type] += 1
            else:
                operators[node.type] = 1 
            
        if node.name:
            traverse(node.name)
        if node.field:
            traverse(node.field)
        
    elif isinstance (node, c_ast.Switch):
        print("switch")
        # Switch[cond*, stmt*] 
        
        if "switch" in operators:
            operators["switch"] += 1
        else:
            operators["switch"] = 1  
                
        if '{' in operators:
                operators['{'] += 1
        else:
            operators['{'] = 1  
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1 
            
        if node.cond:
            traverse(node.cond)
        if node.stmt:
            traverse(node.stmt)
        
    elif isinstance (node, c_ast.TernaryOp):
        print("ternary operator")
        # TernaryOperator[cond*, iftrue*, iffalse*]
        
        if "?:" in operators:
            operators["?:"] += 1
        else:
            operators["?:"] = 1 
        
        if node.cond:
            traverse(node.cond)
        if node.iftrue:
            traverse(node.iftrue)
        if node.iffalse:
            traverse(node.iffalse)
        
    elif isinstance (node, c_ast.TypeDecl):
        print("Type Decl")
        # TypeDecl[declname, quals, type*]
        
        # node.declname nije bitan jer je u pitanju deklaracija
        
        '''
        # izgleda da nema potrebe da se i ovde nalazi jer se nalazi u Decl
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print("******************  "+node.quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
        '''
        if node.type:
            traverse(node.type)

    elif isinstance (node, c_ast.Typedef):
        print("Typedef")
        # Typedef[name, quals, storage, type*]
        
        # neka promenljiva kao npr is_typedef ??
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                #print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print(quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                
        # extern, register
        storage = node.storage        
        if storage: # if true
            if (len(storage) == 2):
                #print(storage[0]+"   "+storage[1])
                if (storage[0] in operators):
                    operators[storage[0]] += 1
                else:
                    operators[storage[0]] = 1
                if (storage[1] in operators):
                    operators[storage[1]] += 1
                else:
                    operators[storage[1]] = 1
            else: #(len(storage) == 1):
                #print(storage[0])
                if (storage[0] in operators):
                    operators[storage[0]] += 1
                else:
                    operators[storage[0]] = 1   
                    
        if node.type:
            traverse(node.type)
            
        if ';' in operators:
                operators[';'] += 1
        else:
            operators[';'] = 1  
        
        # name ne bi trebalo da se racuna u deklaraciji
        
    elif isinstance (node, c_ast.Typename):
        print("Typename")
        # Typename[name, quals, type*]
        
        '''
        Typename <params[0]>: name=None, quals=[]
            TypeDecl <type>: declname=None, quals=[]
              IdentifierType <type>: names=['void']
        '''
        
               
        name = node.name
        if name:
            if name in operands:
                operands[name] += 1
            else:
                operands[name] = 1
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print("******************  "+node.quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                    
        #print("**** "+str(type(node.type)))
        
              
        # !!! GRESKA
        if node.type:
        #    print("!!! "+str(type(node.type)))
            traverse(node.type)            
        
    elif isinstance (node, c_ast.UnaryOp):
        print("unary operator")
        # UnaryOperator[op, epxr*]
        
        if node.op:
            if node.op in operators:
                operators[node.op] += 1
            else:
                operators[node.op] = 1
                
        if node.expr:
            traverse(node.expr)
        
    elif isinstance (node, c_ast.Union):
        print("Union")  
        # Union[name, decls**]
        '''
        union Data {
           int i;
           float f;
           char str[20];
        };
        '''
        
        if "union" in operators:
            operators["union"] += 1
        else:
            operators["union"] = 1            
        if '{' in operators:
                operators['{'] += 1
        else:
            operators['{'] = 1  
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1             
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1 
            
        # name ne bi trebalo da se racuna u deklaraciji
        
        if node.decls:
            for decl in node.decls:
                traverse(decl)
                
    elif isinstance (node, c_ast.While):
        print("while")     
        # While: [cond*, stmt*]
        
        if "while" in operators:
            operators["while"] += 1
        else:
            operators["while"] = 1            
        if '{' in operators:
                operators['{'] += 1
        else:
            operators['{'] = 1  
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1    
        
        if node.cond:
            traverse(node.cond)
        if node.stmt:
            traverse(node.stmt)
        
    else:
        print("Error!")
        return 
    
    return

In [58]:
# FileAst[ext**]
for node in ast.ext:
    print(node)
    traverse(node)

Typedef
Type Decl
enum
Enumerator list
Enumerator: DODAJ AKCIJE
Enumerator: DODAJ AKCIJE
Typedef
Type Decl
struct
Decl
PtrDecl
Type Decl
Identifier Type
Decl
PtrDecl
Type Decl
Identifier Type
Typedef
Type Decl
struct
Decl
PtrDecl
Type Decl
Identifier Type
Decl
PtrDecl
Type Decl
struct
Typedef
Type Decl
struct
Decl
Type Decl
Identifier Type
Decl
PtrDecl
PtrDecl
Type Decl
Identifier Type
FuncDef
Decl: name=hash_func, quals=[], storage=['static'], funcspec=[]
  FuncDecl <type>: 
    ParamList <args>: 
      Decl <params[0]>: name=str, quals=['const'], storage=[], funcspec=[]
        PtrDecl <type>: quals=[]
          TypeDecl <type>: declname=str, quals=['const']
            IdentifierType <type>: names=['char']
      Decl <params[1]>: name=table_size, quals=[], storage=[], funcspec=[]
        TypeDecl <type>: declname=table_size, quals=[]
          IdentifierType <type>: names=['unsigned', 'int']
    TypeDecl <type>: declname=hash_func, quals=[]
      IdentifierType <type>: names=['unsig

ID
ID
Func call
ID
Expr list
ID
}
-----------------


In [59]:
print(operands)
print(operators)

{'char *': 6, 'Entry *': 3, 'unsigned int *': 1, 'Node **': 1, 'unsigned int ': 11, 'hash_value': 4, 'str': 3, 'a': 1, 'table_size': 8, 'Hash **': 1, 'ReturnCode ': 3, 'FAIL': 5, 'hash': 25, 'malloc': 6, 'NULL': 14, 'heads': 16, 'i': 10, 'SUCCESS': 3, 'Hash *': 5, 'hash_func': 3, 'Node *': 7, 'HashRemove': 1, 'entry': 21, 'key': 12, 'index': 10, 'Node ': 1, 'Entry ': 1, 'strlen': 2, 'value': 7, 'strcpy': 2, 'next': 8, 'temp': 17, 'strcmp': 2, 'temp1': 12, 'temp2': 3, 'free': 10, 'char **': 1, 'void ': 3, 'PrintFunc': 1}
{'typedef': 4, 'enum': 1, ';': 75, '{': 38, '}': 38, 'struct': 4, 'static': 1, 'const': 6, '=': 34, 'for': 4, '!=': 5, '*': 12, '++': 4, '%': 1, '+': 3, 'return': 13, 'if': 9, '<': 4, '==': 7, 'sizeof': 4, '->': 69, ',': 9, 'while': 4, '!': 2, 'else': 1, '||': 1}
